In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD license
python 3
"""
import numpy as np

In [2]:
## reference page [The Unreasonable Effectiveness of Recurrent Neural Networks]
## http://karpathy.github.io/2015/05/21/rnn-effectiveness/
## this is a 3 layers neuron network.
## input layer: one hot vector, dim: [vocab * 1]
## hidden layer: LSTM, hidden vector: [hidden_size * 1]
## output layer: Softmax, vocab * 1, the probabilities distribution of each character

In [3]:
# data I/O
data = open('input.txt', 'r').read()

# use set() to count the vacab size
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
print(repr(''.join(sorted(str(x) for x in chars))))

data has 1599 characters, 52 unique.
"\n ',.:;ABCDEGIKLMNOPRSTUVWYabcdefghijklmnoprstuvwxy’"


In [4]:
# dictionary to convert char to idx, idx to char
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i,ch in enumerate(chars)}

In [5]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [6]:
# model parameters
## RNN/LSTM
## this is not LSTM, is the simple basic RNN
## # update the hidden state
## self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
## # compute the output vector
## y = np.dot(self.W_hy, self.h)
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden (100,25)
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden (100,100)
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output (25,100)
bh = np.zeros((hidden_size, 1)) # hidden bias (100,1)
by = np.zeros((vocab_size, 1)) # output bias (25,1)

In [7]:
## compute loss, derivative
## cross-entropy loss is used
## actually, here the author use cross-entropy as error,
## but in the backpropagation the author use sum of squared error (Quadratic cost) to do back propagation.
## be careful about this trick. 
## this is because the output layer is a linear layer.
## TRICK: Using the quadratic cost when we have linear neurons in the output layer, z[i] = a[i]

In [49]:
def lossFun(inputs, targets, hprev):
    """
        inputs, targets are both list of integers.
        hprev is Hx1 array of initial hidden state
        returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {} # s := state
    ## record each hidden state of
    hs[-1] = np.copy(hprev) # hs is dict {-1: (100, 1)}
    loss = 0
    
    # forward pass for each training data point
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1)) # encode in 1-of-k(one-hot) representation
        xs[t][inputs[t]] = 1 # 해당하는 글자에만 값을 1로 설정 - [0, ..., 0, 1, 0, ..., 0]
        # x[0][one_hot_vector], x[1][one_hot_vector], x[2][one_hot_vector], ...
        
        ## hidden state, using previous hidden state hs[t-1]
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
        ## unnormalized log probabilities for next chars, 다음 글자가 어떤 글자가 나올지에 가능성을 표시한 array(정규화되지 않음)
        ys[t] = np.dot(Why, hs[t]) + by
        ## probabilities for next chars, softmax
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # ps[t].shape is (vocab_sizem, 1)
        ## softmax (cross-entropy loss)
        loss += -np.log(ps[t][targets[t], 0])
        
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))): # t=24(=seq_length-1)부터 시작
        ## compute derivative of error w.r.t the output probabilities
        ## dE/dy[j] = y[j] - t[j]
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y
    
        ## output layer does not use activation function, so no need to compute the derivative of error with regard to the net input
        ## of output layer.
        ## then, we could directly compute the derivative of error with regard to the weight between hidden layer and output layer.
        ## dE/dy[j]*dy[j]/dWhy[j,k] = dE/dy[j] * h[k]
        dWhy += np.dot(dy, hs[t].T)
        dby += dy

        ## backprop into h
        ## derivative of error with regard to the output of hidden layer
        ## derivative of H, come from output layer y and also come from H(t+1), the next time H
        dh = np.dot(Why.T, dy) + dhnext
        
        ## backprop through tanh nonlinearity
        ## derivative of error with regard to the input of hidden layer
        ## dtanh(x)/dx = 1 - tanh(x) * tanh(x)
        dhraw = (1 - hs[t] * hs[t]) * dh
        dbh += dhraw
        
        ## derivative of the error with regard to the weight between input layer and hidden layer
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        ## derivative of the error with regard to H(t+1)
        ## or derivative of the error of H(t-1) with regard to H(t)
        dhnext = np.dot(Whh.T, dhraw)
        
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [50]:
## given a hidden RNN state, and a input char id, predict the coming n chars
def sample(h, seed_ix, n):
    """
        sample a sequence of integers from the model
        h is memory state, seed_ix is seed letter for first time step
        지정된 글자 수(n) 만큼의 글자 결과값(숫자의 리스트)을 출력
        h 는 hidden state, seed_ix는 주어진 첫번째 글자
    """
    ## a one-hot vector
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    
    ixes = []
    for t in range(n):
        ## self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        ## y = np.dot(self.W_hy, self.h)
        y = np.dot(Why, h) + by
        ## softmax
        p = np.exp(y) / np.sum(np.exp(y))
        ## sample according to probability distribution
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        
        ## update input x
        ## use the new sampled result as last input, then predict next char again.
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        
        # 결과값 리스트에 추가
        ixes.append(ix)
    return ixes

In [51]:
## iterator counter
n = 0
## data pointer
p = 0

# memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) 
# loss at iteration 0
smooth_loss = -np.log(1.0/vocab_size)*seq_length

In [52]:
## main loop
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    # 데이터를 모두 사용하면 입력 데이터의 맨 처음으로 이동
    if p + seq_length + 1 >= len(data) or n == 0:
        # reset RNN memory
        hprev = np.zeros((hidden_size, 1)) ## hprev is the hiddden state of RNN
        p = 0 # go from start of data
    
    # 입력(p~p+24번째 글자), 목표(p+1~p+25번째 글자) 데이터를 준비 
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    
    # 학습을 100번 반복할 때마다 학습 결과를 출력
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 30) #지금까지 학습한 RNN을 이용하여 숫자의 리스트를 출력
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))
    
    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    ## author using Adagrad(a kind of gradient descent)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: 
        print('iter %d, loss: %f' % (n, smooth_loss)) # 반복횟수, 손실 출력
        
    # perform parameter update with Adagrad
    ## parameter update for Adagrad is different from gradient descent parameter update
    ## need to learn what is Adagrad exactly is.
    ## seems using weight matrix, derivative of weight matrix and a memory matrix, update memory matrix each iteration
    ## memory is the accumulation of each squared derivatives in each iteration.
    ## mem += dparam * dparam
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], # 가중치
                                  [dWxh, dWhh, dWhy, dbh, dby], # 그래디언트
                                  [mWxh, mWhh, mWhy, mbh, mby]): # 메모리
        mem += dparam * dparam
        ## learning_rate is adjusted by mem, if mem is getting bigger, then learning_rate will be small
        ## gradient descent of Adagrad
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update 
    p += seq_length # move data pointer by length of seq_length
    n += 1 # iteration counter 

----
 ANDA'Airsouth thesbith he hoy, 
----
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
iter 0, loss: 98.736031
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17

5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


KeyboardInterrupt: 